In [1]:
import tensorflow as tf

In [2]:
from cifar_data import get_file_queue

In [3]:
q = get_file_queue()

In [6]:
s = q.dequeue()

In [ ]:
sess = tf.Session()
s.eval(session=sess)

In [4]:
tf.train.string_input_producer?